In [44]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase

load_dotenv()

True

## Load important keys

In [47]:
AURA_INSTANCENAME = os.environ["AURA_INSTANCENAME"]
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE = os.environ["NEO4J_DATABASE"]
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

In [49]:
driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH, database=NEO4J_DATABASE)


In [51]:
## Just to test whether connection with database is successful or not
def connect_and_query():
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result = session.run("MATCH (n) RETURN count(n)")
            count = result.single().value()
            print(f"number of nodes: {count}")
    except Exception as e:
            print(f"Error : {e}")
    #finally:
            #driver.close()

In [53]:
connect_and_query()

number of nodes: 5


In [55]:
def createEntities(tx):
        # Create Albert Einstein node
    tx.run("MERGE (a:Person {name: 'Albert Einstein'})")

    # Create other nodes
    tx.run("MERGE (p:Subject {name: 'Physics'})")
    tx.run("MERGE (n:NobelPrize {name: 'Nobel Prize in Physics'})")
    tx.run("MERGE (g:Country {name: 'Germany'})")
    tx.run("MERGE (u:Country {name: 'USA'})")

    

In [57]:
def createRelationships(tx):
        # Create studied relationship
    tx.run(
        """
    MATCH (a:Person {name: 'Albert Einstein'}), (p:Subject {name: 'Physics'})
    MERGE (a)-[:STUDIED]->(p)
    """
    )

    # Create won relationship
    tx.run(
        """
    MATCH (a:Person {name: 'Albert Einstein'}), (n:NobelPrize {name: 'Nobel Prize in Physics'})
    MERGE (a)-[:WON]->(n)
    """
    )

    # Create born in relationship
    tx.run(
        """
    MATCH (a:Person {name: 'Albert Einstein'}), (g:Country {name: 'Germany'})
    MERGE (a)-[:BORN_IN]->(g)
    """
    )

    # Create died in relationship
    tx.run(
        """
    MATCH (a:Person {name: 'Albert Einstein'}), (u:Country {name: 'USA'})
    MERGE (a)-[:DIED_IN]->(u)
    """
    )

In [59]:
def build_knowledge_graph():
    # Open a session with the Neo4j database

    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            # Create entities
            session.execute_write(createEntities)
            # Create relationships
            session.execute_write(createRelationships)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [61]:
build_knowledge_graph()

In [63]:
def query_graph(cypher_query):
    driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result = session.run(cypher_query)
            for record in result:
                #print(record["name"])
                print(record["path"])
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [69]:
einstein_query = """
MATCH path=(a:Person {name: 'Albert Einstein'})-[:STUDIED]->(s:Subject)
RETURN path
UNION
MATCH path=(a:Person {name: 'Albert Einstein'})-[:WON]->(n:NobelPrize)
RETURN path
UNION
MATCH path=(a:Person {name: 'Albert Einstein'})-[:BORN_IN]->(g:Country)
RETURN path
UNION
MATCH path=(a:Person {name: 'Albert Einstein'})-[:DIED_IN]->(u:Country)
RETURN path
"""


# Simple Cypher query to find all node names
simple_query = """
MATCH (n)
RETURN n.name AS name
"""

In [71]:
query_graph(einstein_query)

<Path start=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:5' labels=frozenset({'Person'}) properties={'name': 'Albert Einstein'}> end=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:4' labels=frozenset({'Subject'}) properties={'name': 'Physics'}> size=1>
<Path start=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:5' labels=frozenset({'Person'}) properties={'name': 'Albert Einstein'}> end=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:6' labels=frozenset({'NobelPrize'}) properties={'name': 'Nobel Prize in Physics'}> size=1>
<Path start=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:5' labels=frozenset({'Person'}) properties={'name': 'Albert Einstein'}> end=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:2' labels=frozenset({'Country'}) properties={'name': 'Germany'}> size=1>
<Path start=<Node element_id='4:9b2d7c87-c240-4a14-b823-66c4c1fc673b:5' labels=frozenset({'Person'}) properties={'name': 'Albert Einstein'}> end=<Node elemen